# **Task11**

Use the code from: https://medium.com/@rekalantar/pytorch-tutorial-dynamic-weight-pruning-for-more-optimized-and-faster-neural-networks-7b337e47987b to do dynamic weight pruning. If the full dataset is too large use a suitable subset size.

Use post-training static quantization as well as dynamic quantization as given in: https://pytorch.org/tutorials/recipes/quantization.html to generate a dynamic weight pruning+quantized model. Compare accuracy of the quantized model vs the non-quantized model.


## Importing the required Libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchsummary
from torchvision import models, datasets
from timeit import default_timer as timer
import torch.quantization as quant
import os

Checking for device

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Loading the CIFAR10 dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# Load and subset the dataset
full_dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(full_dataset, batch_size=100, shuffle=True)
# Load the test dataset
test_dataset = datasets.CIFAR10('./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

Files already downloaded and verified


# Creating a Model

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, input_channels, out_planes, stride= 1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, out_planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 =  nn.Conv2d(out_planes, out_planes, kernel_size=3,padding=1)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.downsample = downsample
        # FloatFunction()
        self.skip_add = nn.quantized.FloatFunctional()
        

    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample is not None:
            identity = self.downsample(identity)
        # Notice the addition operation in both scenarios
        x  = self.skip_add.add(x, identity)
        x = self.relu(x)
        
        return x


class ResNet(nn.Module):
    
    def __init__(self, block=BasicBlock, layers=[2, 2, 2, 2], num_channels =3 ,num_classes=10):
        super(ResNet, self).__init__()
        self.input_channels = 64
        self.conv1 = nn.Conv2d(num_channels, self.input_channels, kernel_size = 7, stride = 2, padding = 3)
        self.bn1 = nn.BatchNorm2d(self.input_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.layer1 = self._make_layer(block, layers[0], out=64,  stride= 1)
        self.layer2 = self._make_layer(block, layers[1], out=128, stride= 2)
        self.layer3 = self._make_layer(block, layers[2], out=256, stride= 2)
        self.layer4 = self._make_layer(block, layers[3], out=512, stride= 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def _make_layer(self, block, residual_blocks ,out, stride):
        downsample = None
        layers = []
        if stride != 1 or self.input_channels != out:
            downsample = nn.Sequential(nn.Conv2d(self.input_channels, out, kernel_size=1, stride= stride),
                                       nn.BatchNorm2d(out))
        layers.append(block(self.input_channels, out, stride, downsample))
        self.input_channels = out
        for _ in range(1, residual_blocks):
            layers.append(block(self.input_channels, out))

        return nn.Sequential(*layers)

    
    def forward(self, x):
        # Input are quantized
        x = self.quant(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.dequant(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x

Calling the model ResNet()

In [5]:
model = ResNet()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

### Defining the optimizer and loss function

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### Function to count the zero parameters in the model

In [7]:
def count_zero_params(model):
    zero_count = 0
    for param in model.parameters():
        zero_count += torch.sum(param == 0).item()
    return zero_count

Printing the No.of Zero parameters

In [10]:
before_pruning = count_zero_params(model)
print(before_pruning)

4801


Printing the weights of the model

In [11]:
print("Original weights:\n", model.conv1.weight)

Original weights:
 Parameter containing:
tensor([[[[ 0.0517, -0.0364,  0.0254,  ...,  0.0388,  0.0648,  0.0686],
          [ 0.0620,  0.0512,  0.0271,  ..., -0.0539,  0.0640,  0.0584],
          [-0.0240, -0.0582,  0.0729,  ..., -0.0776,  0.0810,  0.0155],
          ...,
          [ 0.0545, -0.0712, -0.0585,  ..., -0.0130, -0.0214,  0.0099],
          [-0.0768, -0.0390, -0.0125,  ..., -0.0373,  0.0256, -0.0721],
          [ 0.0303,  0.0219, -0.0359,  ..., -0.0336,  0.0306,  0.0317]],

         [[-0.0561, -0.0357, -0.0672,  ...,  0.0365,  0.0232,  0.0119],
          [-0.0164, -0.0330,  0.0526,  ...,  0.0348, -0.0446,  0.0369],
          [-0.0603, -0.0396, -0.0579,  ...,  0.0194,  0.0288, -0.0253],
          ...,
          [ 0.0471,  0.0602, -0.0717,  ...,  0.0816, -0.0549,  0.0257],
          [-0.0797, -0.0068,  0.0176,  ...,  0.0160, -0.0575, -0.0629],
          [ 0.0383, -0.0182,  0.0418,  ..., -0.0396,  0.0313, -0.0437]],

         [[-0.0416,  0.0784,  0.0807,  ...,  0.0183,  0.0168,

Function to train the model and to do weight pruning

In [12]:
def train_pruned_model(model, device, train_loader, optimizer, criterion,epochs,prune_epoch):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss : {running_loss / len(train_loader)}')       
        
        if (epoch+1) % prune_epoch == 0:
            prune_model(model, pruning_rate=0.1)
            print('Pruning done.')

In [13]:
import torch.nn.utils.prune as prune

def prune_model(model, pruning_rate=0.1):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            
            # Applying unstructured L1 norm pruning
            prune.l1_unstructured(module, name='weight', amount=pruning_rate)
            
            prune.remove(module, 'weight')

Function to Evaluate the model

In [14]:
import time
def evaluate_model(model,device):
    model = model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            st = time.time()
            outputs = model(images)
            et = time.time()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct // total} %')
    print('Elapsed time = {:0.4f} milliseconds'.format((et - st) * 1000))
    print("====================================================================================================")

## Calling the function train_model to train 

In [15]:
train_pruned_model(model, device, train_loader, optimizer,criterion, epochs=15,prune_epoch=5)

Epoch [1/15], Loss : 1.4925556901693344
Epoch [2/15], Loss : 1.0655402550697326
Epoch [3/15], Loss : 0.8518562903404235
Epoch [4/15], Loss : 0.7238209476470947
Epoch [5/15], Loss : 0.5991190549135208
Pruning done.
Epoch [6/15], Loss : 0.5140401971936226
Epoch [7/15], Loss : 0.4294527492821217
Epoch [8/15], Loss : 0.3498104492723942
Epoch [9/15], Loss : 0.29040762688219546
Epoch [10/15], Loss : 0.2406279271543026
Pruning done.
Epoch [11/15], Loss : 0.1925336010158062
Epoch [12/15], Loss : 0.1523590096011758
Epoch [13/15], Loss : 0.13485703682899475
Epoch [14/15], Loss : 0.11096452654153109
Epoch [15/15], Loss : 0.09422422910109163
Pruning done.


Printing the weights after weight pruning

In [16]:
print("Pruned weights:\n", model.conv1.weight)

Pruned weights:
 Parameter containing:
tensor([[[[ 0.0283, -0.0000,  0.0969,  ...,  0.2377,  0.2719,  0.2704],
          [ 0.0937,  0.1755,  0.2025,  ...,  0.3112,  0.4571,  0.3884],
          [ 0.0378,  0.0160,  0.1953,  ...,  0.1639,  0.3054,  0.2367],
          ...,
          [-0.0850, -0.2339, -0.2266,  ..., -0.2001, -0.1158, -0.0329],
          [-0.1494, -0.1883, -0.2235,  ..., -0.1210, -0.0259, -0.0951],
          [-0.0800, -0.0899, -0.1976,  ..., -0.1581, -0.1125, -0.0542]],

         [[-0.0981, -0.0684, -0.1841,  ..., -0.0808, -0.1230, -0.0682],
          [-0.1321, -0.1509, -0.1528,  ..., -0.1558, -0.2710, -0.1610],
          [-0.1006, -0.1349, -0.1860,  ..., -0.0967, -0.1624, -0.1573],
          ...,
          [ 0.0394,  0.0785, -0.0386,  ...,  0.0206, -0.0891, -0.0199],
          [ 0.0482,  0.1054,  0.0805,  ...,  0.1334,  0.0386,  0.0142],
          [ 0.1483,  0.1180,  0.1499,  ...,  0.0794,  0.0826,  0.0219]],

         [[ 0.0000,  0.1675,  0.0580,  ..., -0.0747, -0.1321, -

No.of Zero parameters after weight pruning

In [17]:
after_pruning = count_zero_params(model)
print(after_pruning)

1117203


Evaluating the model and printing the accuracy

In [18]:
device ='cpu'
print("=====================================AFTER WEIGHT PRUNING========================================")
evaluate_model(model,device)

=====================================AFTER WEIGHT PRUNING========================================
Accuracy: 75 %
Elapsed time = 31.1096 milliseconds


Function to print the size of the model

In [19]:
def print_size_of_model(model,path):
    torch.save(model.state_dict(), path)
    print('Size (MB):', os.path.getsize(path)/1e6)

Printing the size of the model before quantization 

In [20]:
print("=====================================BEFORE QUANTIZATION========================================")
print(model)
print('Size of the model Before Dynamic quantization')
print_size_of_model(model,path="FinalTask11.pt")

=====================================BEFORE QUANTIZATION========================================
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddin

# Dynamic quantization

Doing dynamic quantization for the model

In [21]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)

Printing the model and size of it after dynamic quantization

In [22]:
print("=====================================DYNAMIC QUANTIZATION========================================")
print(model_dynamic_quantized)
print('Size of the model After Dynamic quantization')
print_size_of_model(model_dynamic_quantized,path="FinalTask11_dynamic.pt")

=====================================DYNAMIC QUANTIZATION========================================
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddi

Evaluating and printing the accuracy of the model after dynamic quantization

In [23]:
device ='cpu'
print("=====================================ACCURACY AFTER DYNAMIC QUANTIZATION=======================================")
evaluate_model(model_dynamic_quantized,device)

=====================================ACCURACY AFTER DYNAMIC QUANTIZATION=======================================
Accuracy: 76 %
Elapsed time = 26.0303 milliseconds


# Static Quantization

Doing Static quantization for the model

In [25]:
model_dynamic_quantized.eval()
model_dynamic_quantized.qconfig = torch.quantization.get_default_qconfig('fbgemm')
static_quantized = torch.quantization.prepare(model_dynamic_quantized, inplace=True)
print(static_quantized)
with torch.no_grad():
    for data, target in train_loader:
        static_quantized(data)
static_quantized = torch.quantization.convert(static_quantized, inplace=True)
print(static_quantized)

ResNet(
  (conv1): QuantizedConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.09044206887483597, zero_point=63, padding=(3, 3))
  (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): QuantizedReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.2592632472515106, zero_point=69, padding=(1, 1))
      (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): QuantizedReLU(inplace=True)
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.12315140664577484, zero_point=75, padding=(1, 1))
      (bn2): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): QFunctional(
        scale=0.1417912095785141, zero_point=42
        

In [52]:
import io
# save/load using scripted model
# scripted = torch.jit.script(static_quantized)
# Static = io.BytesIO()
# torch.jit.save(scripted, Static)
# print(scripted)
# Save
torch.jit.save(torch.jit.script(static_quantized), "static_.pth")
# Load
# mq = torch.jit.load("quant_model.pth")
# b.seek(0)
# scripted_quantized = torch.jit.load(b)

In [46]:
def print_size_of_quantized_model(path):
    print('Size (MB):', os.path.getsize(path)/1e6)

Printing the model and size of it after Static quantization

In [53]:
print('Size of the model After Static quantization')
print_size_of_quantized_model(path="static_.pth")

Size of the model After Static quantization
Size (MB): 11.429824


Evaluating and finding the accuracy of the model 

In [54]:
device ='cpu'
print("=====================================ACCURACY AFTER STATIC QUANTIZATION=======================================")
evaluate_model(static_quantized,device)

=====================================ACCURACY AFTER STATIC QUANTIZATION=======================================
Accuracy: 76 %
Elapsed time = 9.9797 milliseconds
